In [325]:
import numpy as np
import math
np.set_printoptions(edgeitems=4,formatter={'float_kind':'{:0.8f}'.format})

def M(X,Y):
    result=np.zeros((len(X),len(Y[0])))
    if( len(X[0])==len(Y) ):
        for i in range(len(X)):
            for j in range(len(Y[0])):
                for k in range(len(Y)):
                     result[i][j] += X[i][k] * Y[k][j]
    elif(len(X[0])!=len(Y)):
        result=0
    return(result) 

def em(b):
    a=[]
    for i in range(0,b):
        a.append([])
    return(a)

def zm(l):
    a=[]
    #delta=[]
    for i in range(1,len(l)):
        a.append(np.zeros(shape=( len(l[i].delta),len(l[i-1].out) ) ))
    return a   
    
class i:
    def __init__(self,inp,nx):
        self.inp=inp
        self.out=em(len(inp)+1)
        self.wei=np.random.random((nx,len(inp)+1))
        for i in range(0,len(inp)+1):
                if(i==0):
                    self.out[i].append(1)
                elif(i!=0):
                    self.out[i].append(inp[i-1])
        self.nz=M(self.wei,self.out)        


    
class layer:
    def __init__(self,inp,nx):
        self.inp=inp
        self.nx=nx
        self.wei=np.random.random((self.nx,len(inp.nz)+1))
        #self.nodes=[]
        self.z=inp.nz
        self.out=em(len(inp.nz)+1)
        self.dout=em(len(inp.nz)+1)
        self.delta=[]
        for i in range(0,len(inp.nz)+1):
            if(i==0):
                self.out[i].append(1)
                self.dout[i].append(self.out[i][0]*(1-self.out[i][0]))
            elif(i!=0):   
                self.out[i].append(1/(1+math.exp(-1*self.z[i-1][0])))
                self.dout[i].append(self.out[i][0]*(1-self.out[i][0]))
        self.nz=M(self.wei,self.out)          
        
class output:
    def __init__(self,inp,ops):
        self.out=[]
        self.delta=[]
        self.z=inp.nz
        self.ops=ops
        self.delta=em(len(inp.nz))
        self.out=em(len(inp.nz))    
        for i in range(0,len(inp.nz)):
            self.out[i].append(1/(1+math.exp(-1*self.z[i][0])))        
            self.delta[i].append(self.out[i][0]-ops[0])
            


def backdelta(i):
    a=M(l[i].wei.T,l[i+1].delta)
    s=em(len(a))   
    for j in range(0,len(s)):
        s[j].append(l[i].dout[j][0]*a[j][0])
    del s[0]
    return(s)

def backprop(l):
    a=[]
    for i in range(0,len(l)-2):
        l[len(l)-2-i].delta=backdelta(len(l)-2-i)
    for j in range(1,len(l)):
        a.append(l[j].delta)
    return(a)   

def d(l):
    a=zm(l)
    for i in range(0,len(l)-1):
        b=np.asmatrix(l[i+1].delta)
        c=np.asmatrix(l[i].out).T
        a[i]=a[i] + b*c 
    return(a)


def D(l,a):
    b=zm(l)
    c=d(l)
    for i in range(0,len(b)):
         for j in range(0,len(b[i])):
                for k in range(0,len(b[i][0])):
                    if(k!=0):
                        b[i][j][k]=(c[i][j,k]+ a*l[i].wei[j,k])/len(l[0].inp)
                    if(k==0):
                        b[i][j][k]=c[i][j,k]/len(l[0].inp)

    return(b)

class network:
    def __init__(self,data,nl):
        self.data=data
        self.ip=data[0]
        self.ot=data[1]
        self.nl=nl
        self.lrs=[]
        self.lrs.append(i(data[0],nl[0]))         #5
        for j in range(1,len(nl)):
            self.lrs.append(layer(self.lrs[j-1],nl[j]))
        self.lrs.append(layer(self.lrs[len(nl)-1],len(self.ot))) 
        self.lrs.append(output(self.lrs[len(nl)],self.ot))

In [326]:
l=[]
data=[[1,0,1,0,1],
      [1]]
nl=[4,3,2]
a=network(data,nl)
l=a.lrs
b=backprop(l) 

In [327]:
D(l,0.01)

[array([[-0.00002868, 0.00183751, 0.00070053, 0.00029881, 0.00181542,
         0.00019737],
        [-0.00001072, 0.00195241, 0.00050342, 0.00157226, 0.00196875,
         0.00149816],
        [-0.00001328, 0.00165176, 0.00015674, 0.00097840, 0.00109870,
         0.00128729],
        [-0.00003579, 0.00067566, 0.00130707, 0.00087305, 0.00159408,
         0.00091672]]),
 array([[-0.00025869, 0.00122175, 0.00086820, 0.00106211, 0.00089266],
        [-0.00005368, 0.00105199, 0.00175478, 0.00122823, 0.00144940],
        [-0.00007112, 0.00098912, 0.00028111, 0.00106242, 0.00111549]]),
 array([[-0.00242388, -0.00024530, -0.00146553, -0.00186563],
        [-0.00290762, -0.00178925, -0.00264883, -0.00231724]]),
 array([[-0.02633117, -0.02158400, -0.01872007]])]